In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm
import statsmodels.api as sm
path = r"Proposal/Figures/"
table_path = r"Proposal/Tables/"

C:\Users\SE.5203\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [3]:
def function_z_k(α,γ,r_b, green_premium,τ_E,A_tilde):
    r_g = (1-green_premium) * r_b
    return (α/(1-α) * (r_b +τ_E/1000*A_tilde )/r_g) ** (1/γ)
def function_z_l(z_k,β,α,γ,w,green_premium,r_b):
    r_g = (1-green_premium) * r_b
    return (((1-β)/β) /(α) )* ((α + (1-α)*(z_k** (1-γ)) ) ** (1/(1-γ))) * (r_g/w)

def function_intensity(A_tilde,A_hat,α,γ,z_l,z_k,β):
    return (A_tilde/A_hat) * (α * (z_k ** (γ-1)) + (1 - α)) ** (γ / (1-γ)) * (z_l ** (1-β))

def function_price(A_tilde,A_hat,α,γ,z_l,z_k,β,w,green_premium,r_b,σ,τ_E):
    r_g = (1-green_premium) * r_b
    C_G = r_g * (((α + (1-α)*(z_k** (1-γ)) ) ** (γ / (1-γ))) * (z_l ** (1-β)))
    C_B = r_b * ((α * (z_k ** (γ-1)) + (1 - α)) ** (γ / (1-γ))) * (z_l ** (1-β))
    c_L = w * z_l ** (-β)
    C_E = τ_E/1000 * A_tilde * C_B/r_b
    return σ/(σ-1) * (C_G + C_B + c_L + C_E) / A_hat

    
def production_function(A_hat,β,z_l,L):
    return A_hat * (z_l ** (β)) * L

def brown_capital(A_hat,α,z_k,z_l,γ,β,Y):
    return (Y/A_hat) * ((α * (z_k ** (γ-1)) + (1 - α)) ** (γ / (1-γ))) * (z_l ** (1-β))
def green_capital(A_hat,α,z_k,z_l,γ,β,Y):
    return (Y/A_hat) * ((α + (1-α)*(z_k** (1-γ)) ) ** (γ / (1-γ))) * (z_l ** (1-β))

def simulate_firms(n,A_tilde,A_hat,L,α,γ,r_b, green_premium,τ_E,β,w,σ):
    # simulate over multiple firms
    r_g = (1-green_premium) * r_b
    np.random.seed(0)
    A_tilde_vector = (1 + np.random.lognormal(mean=0,sigma=1,size=n)) * A_tilde
    A_hat_vector = (1 + np.random.lognormal(mean=0,sigma=1,size=n)) * A_hat
    L_vector = (np.random.lognormal(mean=-1.3,sigma=1.8,size=n)) * L
    intensity = []
    production = []
    emissions = []
    emission_cost = []
    G_c = []
    B_c = []
    for i in zip(A_tilde_vector,A_hat_vector,L_vector):
        z_k = function_z_k(α,γ,r_b, green_premium,τ_E,i[0])
        z_l = function_z_l(z_k,β,α,γ,w,green_premium,r_b)
        IN = function_intensity(i[0],i[1],α,γ,z_l,z_k,β)
        Y = production_function(i[1],β,z_l,i[2])
        intensity.append(IN)
        production.append(Y)
        emissions.append(IN * Y)
        emission_cost.append(τ_E/1000 * IN * Y)
        G_c.append(green_capital(i[1],α,z_k,z_l,γ,β,Y))
        B_c.append(brown_capital(A_hat,α,z_k,z_l,γ,β,Y))
    return CES_aggregator(emissions,np.inf),CES_aggregator(production,σ),sum(intensity)/n,production,emission_cost,G_c,B_c
def CES_aggregator(array,σ):
    if σ != np.inf:
        return sum([i ** ((σ-1)/σ) for i in array]) ** (σ/(σ-1))
    else:
        return sum(array)


In [54]:
params = {
    'β': 0.6,
    'γ': 10.34,
    'α': 0.25,
    'A_hat': 0.1,
    'A_tilde': 0.018,
    'L': 250,
    'μ': 0,
    'θ': 1,
    'η': np.inf,
    'σ': np.inf,
    'r_b': 0.05,
    'green_premium': 0,
    'w': 500, # TSEK
    'τ_E': 0, # per ton
    'n': int(1e3)}

# CO2 Emission is 13800 Kton
# Labor 500 TSEK


z_k = function_z_k(params['α'],params['γ'],params['r_b'], params['green_premium'],params['τ_E'],params['A_tilde'])
z_l = function_z_l(z_k,params['β'],params['α'],params['γ'],params['w'],params['green_premium'],params['r_b'])
print("z_k: ",z_k)
print("z_l: ",z_l)
print("intensity: (KG/SEK) ",function_intensity(params['A_tilde'],params['A_hat'],params['α'],params['γ'],z_l,z_k,params['β']))
print("production: (MSEK)",production_function(params['A_hat'],params['β'],z_l,params['L'])/1e3)
print("emission: (ton)",function_intensity(params['A_tilde'],params['A_hat'],params['α'],params['γ'],z_l,z_k,params['β']) * production_function(params['A_hat'],params['β'],z_l,params['L']))
print("-"*50)
emission, production, intensity, _, emission_cost, G_c, B_c = simulate_firms(params['n'],params['A_tilde'],params['A_hat'],params['L'],params['α'],params['γ'],params['r_b'], params['green_premium'],params['τ_E'],params['β'],params['w'],params['σ'])
print("Intensity: (KG/SEK)",intensity)
print("Emission: (MTon) ",emission/1e6)
print("Production: (BSek) ",production/1e6)
print("Emission/Production: (KG/SEK) ",emission/production)
print("Emission cost: ",sum(emission_cost))
print("Green Capital: ",sum(G_c))
print("Brown Capital: ",sum(B_c))

z_k:  0.8992009254377683
z_l:  0.0002442220022023017
intensity: (KG/SEK)  0.007810623157463114
production: (MSEK) 0.00017006340925250108
emission: (ton) 0.0013283012025447117
--------------------------------------------------
Intensity: (KG/SEK) 0.009974165041971773
Emission: (MTon)  2.934341902006241e-06
Production: (BSek)  0.00042075128365303387
Emission/Production: (KG/SEK)  0.006974053356486018
Emission cost:  0.0
Green Capital:  23.50871660909485
Brown Capital:  182.57387331292875


In [79]:
def function_price(A_tilde,A_hat,α,γ,z_l,z_k,β,w,green_premium,r_b,σ,τ_E):
    r_g = (1-green_premium) * r_b
    C_G = r_g * (((α + (1-α)*(z_k** (1-γ)) ) ** (γ / (1-γ))) * (z_l ** (1-β)))
    C_B = r_b * ((α * (z_k ** (γ-1)) + (1 - α)) ** (γ / (1-γ))) * (z_l ** (1-β))
    c_L = w * z_l ** (-β)
    C_E = τ_E/1000 * A_tilde * C_B/r_b
    return σ/(σ-1) * (C_G + C_B + c_L + C_E) / A_hat



def optimal_labor(A_tilde,A_hat,α,γ,z_l,z_k,β,w,green_premium,r_b,σ,τ_E,ν=1e5):
    p = function_price(A_tilde,A_hat,α,γ,z_l,z_k,β,w,green_premium,r_b,σ,τ_E)
    return ν * z_l ** (-β) * (p ** (-σ))/A_hat

def ratios_gen(input_0):
    z_k = function_z_k(input_0['α'],input_0['γ'],input_0['r_b'], input_0['green_premium'],input_0['τ_E'],input_0['A_tilde'])
    z_l = function_z_l(z_k,input_0['β'],input_0['α'],input_0['γ'],input_0['w'],input_0['green_premium'],input_0['r_b'])
    l = optimal_labor(input_0['A_tilde'],input_0['A_hat'],input_0['α'],input_0['γ'],z_l,z_k,input_0['β'],input_0['w'],input_0['green_premium'],input_0['r_b'],input_0['σ'],input_0['τ_E'])
    y = production_function(input_0['A_hat'],input_0['β'],z_l,l)
    p = function_price(input_0['A_tilde'],input_0['A_hat'],input_0['α'],input_0['γ'],z_l,z_k,input_0['β'],input_0['w'],input_0['green_premium'],input_0['r_b'],input_0['σ'],input_0['τ_E'])
    return z_k,z_l,l,y*1e3,p

def gen_df(params,τ_E):
    params_0 = params.copy()
    params_0['σ'] = 1.5
    params_0['w'] = params['w']
    params_0['τ_E'] = τ_E
    params_1 = params_0.copy()
    res_1 = ratios_gen(params_1)
    params_2 = params_0.copy()
    params_2['A_hat'] = params_2['A_hat'] * 0.9
    params_2['A_tilde'] = params_2['A_tilde'] * 1.1
    res_2 = ratios_gen(params_2)
    # put the results into a dataframe
    df = pd.DataFrame([res_1,res_2],columns=['z_k','z_l','l','y','p'])
    # add sum row
    df.loc['sum'] = np.nan
    df.loc['sum','l']= df.l.sum()
    df
    p_s = 0
    y_s = 0
    for i in [res_1,res_2]:
        p_s += i[4] ** (1-params_0['σ'])
        y_s += i[3] ** ((params_0['σ']-1)/(params_0['σ']))
    p_s = p_s ** (1/(1-params_0['σ']))
    y_s = y_s ** (params_0['σ']/(params_0['σ']-1))
    df.loc['sum','p'] = p_s
    df.loc['sum','y'] = y_s
    return df

df_0 = gen_df(params,0)
df_100 = gen_df(params,100)
df = pd.concat([df_0,df_100],keys=['τ_E = 0','τ_E = 100'],axis=0)
df


z_k       z_l         l         y             p
τ_E = 0   0    0.899201  0.000244  0.044895  0.030540  2.205060e+06
          1    0.899201  0.000244  0.042591  0.026076  2.450067e+06
          sum       NaN       NaN  0.087486  0.225992  5.806814e+05
τ_E = 100 0    0.902282  0.000245  0.044936  0.030624  2.201043e+06
          1    0.902585  0.000245  0.042634  0.026154  2.445167e+06
          sum       NaN       NaN  0.087570  0.226641  5.795732e+05